# Busca em profundidade

## 📘 1. Introdução

A Busca em Profundidade é uma estratégia de busca não informada que explora cada ramo da árvore de busca o mais profundamente possível antes de retroceder.

## 📌 Características:

- Estratégia: **LIFO** (último a entrar, primeiro a sair)
- Estrutura de dados: **pilha**
- Completa: Não (em espaços infinitos)
- Ótima: Não
- Complexidade de tempo: $O(b^m)$, onde $b$ é o fator de ramificação e $m$ é a profundidade máxima
- Complexidade de espaço: $O(bm)$

## 📘 2. Estrutura Básica do Problema

### Representação dos nós (estados)

In [1]:
class Estado:
    """
    Representa um estado no espaço de busca de um problema.

    A estrutura é genérica, podendo ser utilizada em problemas como o quebra-cabeça de 8 peças,
    labirintos, jogos e outros domínios que envolvem transições de estado.

    A representação do estado é feita por meio de um vetor unidimensional.
    """

    def __init__(self, vetor=None, custo=0, acao=None, pai=None):
        """
        Inicializa um novo estado.

        Args:
            vetor (list[int], opcional): representação linear do estado (ex: 9 elementos do 8-puzzle).
            custo (int): custo acumulado para atingir este estado a partir do estado inicial.
            acao (str): ação que resultou neste estado a partir do estado pai (ex: 'cima', 'direita').
            pai (Estado): referência ao estado anterior (para reconstrução do caminho da solução).
        """
        if vetor is None:
            vetor = []
        self.vetor = vetor
        self.custo = custo
        self.acao = acao
        self.pai = pai

    def __getitem__(self, i):
        """
        Permite o acesso direto aos elementos do vetor via indexação.

        Args:
            i (int): índice do elemento a ser acessado.

        Returns:
            int: valor na posição i do vetor.
        """
        return self.vetor[i]

    def __eq__(self, outro):
        """
        Define igualdade entre dois estados com base em seus vetores.

        Args:
            outro (Estado): estado a ser comparado.

        Returns:
            bool: True se os vetores forem iguais, False caso contrário.
        """
        if not isinstance(outro, Estado):
            return False
        return self.vetor == outro.vetor

    def __hash__(self):
        """
        Permite o uso de instâncias de Estado como chaves de dicionário ou em conjuntos.

        Returns:
            int: valor hash derivado do vetor (convertido em tupla).
        """
        return hash(tuple(self.vetor))

    def __repr__(self):
        """
        Retorna uma representação legível do vetor do estado, útil para debug.

        Returns:
            str: representação textual do vetor do estado.
        """
        return str(self.vetor)


### Representação do Problema

In [2]:
class Problema:
    """
    Classe base para a modelagem de problemas de busca no espaço de estados.

    Esta classe define a interface e estrutura mínima que deve ser seguida por qualquer
    problema a ser resolvido com algoritmos de busca, como Busca em Largura, A*, entre outros.
    """

    def __init__(self):
        """
        Inicializa o problema com estado inicial nulo.
        Subclasses devem sobrescrever essa inicialização.
        """
        self._estado_inicial = None

    @property
    def estado_inicial(self):
        """
        Retorna o estado inicial do problema.

        Returns:
            Estado: o estado de partida.
        """
        if self._estado_inicial is None:
            raise NotImplementedError("O estado inicial não foi definido.")
        return self._estado_inicial

    def estado_objetivo(self, estado):
        """
        Verifica se o estado fornecido atende à condição de objetivo.

        Args:
            estado (Estado): estado a ser testado.

        Returns:
            bool: True se é um estado objetivo, False caso contrário.
        """
        raise NotImplementedError("Método 'estado_objetivo' deve ser implementado na subclasse.")

    def solucao(self, estado):
        """
        Reconstrói o caminho do estado inicial até o estado objetivo, percorrendo os pais.

        Args:
            estado (Estado): estado objetivo atingido.

        Returns:
            list[Estado]: sequência de estados do caminho da solução.
        """
        resultado = []
        ptr = estado
        while ptr:
            resultado.append(ptr)
            ptr = ptr.pai
        return list(reversed(resultado))

    def funcao_sucessora(self, estado):
        """
        Retorna a lista de estados sucessores (vizinhos) do estado atual,
        aplicando todas as ações válidas possíveis.

        Args:
            estado (Estado): estado atual.

        Returns:
            list[Estado]: estados sucessores gerados.
        """
        raise NotImplementedError("Método 'funcao_sucessora' deve ser implementado na subclasse.")


## 📘 3. Implementação da Busca em Profundidade

In [3]:
def busca_profundidade(problema: Problema):
    """
    Executa o algoritmo de Busca em Profundidade (Depth-First Search - DFS)
    para resolver um problema de busca no espaço de estados.

    Args:
        problema (Problema): Instância que define o problema com os métodos
                             estado_inicial, estado_objetivo e funcao_sucessora.

    Returns:
        list[Estado]: Caminho da solução, do estado inicial ao objetivo.

    Raises:
        RuntimeError: Se a borda for esvaziada sem encontrar uma solução.
    """

    # 1. Inicializa a borda com o estado inicial
    borda = [problema.estado_inicial]  # pilha LIFO

    # 2. Inicializa a memória com o estado inicial visitado
    memoria = [problema.estado_inicial]

    # 3. Loop principal da busca
    while borda:

        # 4. Remove o último estado da borda (topo da pilha)
        estado = borda.pop()

        # 5. Verifica se é o estado objetivo
        if problema.estado_objetivo(estado):
            return problema.solucao(estado)

        # 6. Gera sucessores do estado atual
        vizinhos = problema.funcao_sucessora(estado)

        # 7. Adiciona novos estados ao topo da pilha e à memória
        for vizinho in vizinhos:
            if vizinho not in memoria:
                borda.append(vizinho)
                memoria.append(vizinho)

    # 8. Se a borda esvaziar, a solução não foi encontrada
    raise RuntimeError("Falha ao encontrar solução.")


## 📘 4. Exemplo - Problema das $n$ rainhas

In [4]:
from copy import deepcopy

class ProblemaNRainhas(Problema):
    """
    Problema das n rainhas representado como vetor de posições (linha por coluna).
    """

    def __init__(self, estado_inicial=None):
        """
        Inicializa o problema com um vetor de posições das rainhas.

        Args:
            estado_inicial (list[int], optional): Estado inicial (posição das rainhas nas colunas).
                                                  Se None, usa uma configuração padrão.
        """
        if estado_inicial is None:
            estado_inicial = [0, 1, 2, 3, 4, 5, 6, 7]  # inicialização ingênua (linha = coluna)
        self._estado_inicial = Estado(vetor=estado_inicial, custo=0)

    @property
    def estado_inicial(self):
        return self._estado_inicial

    def estado_objetivo(self, estado):
        """
        Verifica se o estado atual não possui conflitos entre rainhas.

        Args:
            estado (Estado): Estado atual.

        Returns:
            bool: True se não há conflitos entre rainhas.
        """
        return self._conflitos(estado.vetor) == 0

    def funcao_sucessora(self, estado):
        """
        Gera todos os vizinhos mudando uma rainha de linha em uma coluna.

        Args:
            estado (Estado): Estado atual.

        Returns:
            list[Estado]: Estados vizinhos com apenas um movimento por vez.
        """
        sucessores = []
        n = len(estado.vetor)

        for col in range(n):
            for nova_linha in range(n):
                if nova_linha != estado.vetor[col]:
                    novo_vetor = deepcopy(estado.vetor)
                    novo_vetor[col] = nova_linha
                    sucessores.append(
                        Estado(vetor=novo_vetor,
                               custo=estado.custo + 1,
                               acao=f"coluna {col} -> linha {nova_linha}",
                               pai=estado)
                    )

        return sucessores

    def _conflitos(self, vetor):
        """
        Conta o número total de pares de rainhas em conflito.

        Args:
            vetor (list[int]): vetor com a linha de cada rainha por coluna.

        Returns:
            int: número de pares de rainhas em conflito.
        """
        conflitos = 0
        n = len(vetor)
        for i in range(n):
            for j in range(i + 1, n):
                if vetor[i] == vetor[j]:  # mesma linha
                    conflitos += 1
                elif abs(vetor[i] - vetor[j]) == abs(i - j):  # mesma diagonal
                    conflitos += 1
        return conflitos


In [5]:
# Execução do problema
n = 5

problema = ProblemaNRainhas(estado_inicial=[i for i in range(n)])
caminho = busca_profundidade(problema)

# Neste problema o estado final que interessa
estado_final = caminho[-1]

# Impressão simples dos estados no caminho
for linha in range(n):
    linha_str = ''
    for coluna in range(n):
        if caminho[-1][coluna] == linha:
            linha_str += f"♕ "
        else:
            linha_str += f". "
    print(linha_str)
print()

. . . . ♕ 
. . ♕ . . 
♕ . . . . 
. . . ♕ . 
. ♕ . . . 



## 📘 5. Exemplo Visual

In [6]:
from graphviz import Digraph
from IPython.display import display
import time

def busca_profundidade_visual(problema: Problema, atraso=1.0):
    """
    Executa busca em profundidade (DFS) com visualização gráfica da árvore.
    """
    borda = [problema.estado_inicial]
    memoria = [problema.estado_inicial]

    grafo = Digraph(format='png')
    grafo.attr('graph', rankdir='TB')
    grafo.attr('node',
               shape='box',
               style='filled,rounded',
               fontname='monospace',
               fontsize='9',
               width='0.6',
               height='0.6',
               margin='0.1')

    grafo.attr('edge', fontname='monospace', fontsize='10')

    id_nos = {}
    contador_id = 0

    def estado_id(estado):
        nonlocal contador_id
        chave = tuple(estado.vetor)
        if chave not in id_nos:
            id_nos[chave] = f"n{contador_id}"
            contador_id += 1
        return id_nos[chave]

    def formatar_tabuleiro(vetor):
        """Formata vetor de rainhas como string de tabuleiro 8x8."""
        simbolo_rainha = '♕'
        simbolo_vazio = '·'
        linhas = []
        for linha in range(4):
            linha_str = ''
            for coluna in range(4):
                if coluna < len(vetor) and vetor[coluna] == linha:
                    linha_str += simbolo_rainha + ' '
                else:
                    linha_str += simbolo_vazio + ' '
            linhas.append(linha_str)
        return '\n'.join(linhas)

    while borda:
        estado = borda.pop()
        id_estado = estado_id(estado)

        # Estado atual: amarelo
        grafo.node(id_estado, formatar_tabuleiro(estado.vetor), fillcolor='gold')

        # Mostrar grafo parcial
        display(grafo)
        time.sleep(atraso)

        if problema.estado_objetivo(estado):
            return problema.solucao(estado)

        for vizinho in reversed(problema.funcao_sucessora(estado)):  # ordem: esquerda para direita
            id_pai = estado_id(estado)
            id_filho = estado_id(vizinho)

            if vizinho not in memoria:
                grafo.node(id_filho, formatar_tabuleiro(vizinho.vetor), fillcolor='palegreen')
                grafo.edge(id_pai, id_filho, label=vizinho.acao or "")

        for vizinho in problema.funcao_sucessora(estado):  # ordem: esquerda para direita
            if vizinho not in memoria:
                memoria.append(vizinho)
                borda.append(vizinho)

        # Após expansão: cinza escuro
        grafo.node(id_estado, formatar_tabuleiro(estado.vetor), fillcolor='dimgray')

    raise RuntimeError("Falha ao encontrar solução.")


In [7]:
# Descomente o código abaixo para visualizar a execução do algoritmo
n = 4
#problema = ProblemaNRainhas(estado_inicial=[i for i in range(n)])
#caminho = busca_profundidade_visual(problema, atraso=0.8)